In [1]:
from itertools import combinations
from tkinter import *
import pandas as pd

In [2]:
players = ["Hards", "Regly", "Chris", "Lucas", "Antônio", "Filipe"]
font = {"font":("Arial", 11)}
player, *others = players

In [3]:
def show_results(rounds:list, tags:list, qntmatches:int, qnttags:int) -> None:
    results = { p["player"]: { k: v for k, v in p.items() if k != "player" } for p in pd.read_csv("total.csv").to_dict("records") }

    with open("plays.txt", "a") as f:
        f.write(f'{results["Hards"]["round"] + 1}° Round \n' )

        for matches in rounds + tags:
            fullwin, fullloss = { p: 0 for p in players }, { p: 0 for p in players }

            f.write("\n")
            for result in matches:

                p1, p2 = result
                r1, r2 = map(lambda r: int(r.get()), matches[result])

                f.write(f"{p1} {r1} X {r2} {p2}\n")
                if isinstance(p1, tuple):

                    for p in p1: results[p]["points"] += r1
                    for p in p2: results[p]["points"] += r2
                    continue

                if r1 > r2:
                    fullwin[p1] += 1
                    fullloss[p2] += 1
                    results[p1]["wins"] += 1
                    results[p1]["points"] += 3
                    results[p2]["defeats"] += 1

                elif r1 < r2:
                    fullwin[p2] += 1
                    fullloss[p1] += 1
                    results[p2]["wins"] += 1
                    results[p2]["points"] += 3
                    results[p1]["defeats"] += 1

                else:
                    results[p1]["draws"] += 1
                    results[p2]["draws"] += 1
                    results[p1]["points"] += 1
                    results[p2]["points"] += 1

                results[p1]["0wins"] += (r2 == 0)
                results[p2]["0wins"] += (r1 == 0)

            for p in results:
                if fullwin[p] == 5:
                    results[p]["3wins"] += 1
                    results[p]["points"] += 1

                if fullloss[p] == 5:
                    results[p]["reverse"] += 1

        f.write("\n")
        f.write("*" * 50)
        f.write("\n")

    df = pd.DataFrame(results).T.sort_values(["points", "wins", "3wins", "0wins"], ascending=False) \
        .reset_index().rename(columns={"index": "player"})

    df["round"] = df["round"].max() + 1
    df.to_csv("total.csv", index=False)
    df["matches"] = qntmatches
    df["tags"] = qnttags

    pd.concat([pd.read_csv("points.csv"), df]).to_csv("points.csv", index=False)
    print(df.to_string(index=False))

In [4]:
def create_panel(name:str, games:list, root) -> list:
    container = Label(root, name=name)

    for g, game in enumerate(games):
        print(game)
        for m, match in enumerate(game):

            points = []
            for p, player in enumerate(match):
                Label(container, text=" & ".join(player) if isinstance(player, tuple) else player, **font).grid(row=m+(g*10), column=1 if not p else 9)

                point = Entry(container, width=5, )
                point.grid(row=m+(g*10), column=3 if not p else 7)

                points.append(point)
            game[match] = points

            Label(container, text="X", **font).grid(row=m+(g*10), column=5)
    container.grid(columnspan=5)
    return games

In [5]:
def create_count(text:str, root, column=0) -> None:
    Label(root, text=f"{text}: ", **font).grid(row=0, column=column)

    qnt = Spinbox(root, width=5, **font, from_=0, to=3)
    qnt.grid(row=0, column=column+1)
    return qnt

In [6]:
def create_games(qnttags:int, qntmatches:int, root) -> None:

    # tags = create_panel("tags", [{((player, p), tuple(o for o in others if p != o)): () for p in others} for _ in range(qnttags)], root)
    rounds = create_panel("matches", [{ m: () for m in list(combinations(players, 2)) } for _ in range(qntmatches)], root)

    Button(root, text="Save", name="save", command=lambda: show_results(rounds, [], qntmatches, qnttags), width=25, **font).grid(columnspan=5)

In [ ]:
root = Tk()
root.title("Yugioh Championship")

# qnttags = create_count("Tags", root)
qntmatches = create_count("Matches", root, 2)

Button(root, text="Create", command=lambda: create_games(0, int(qntmatches.get()), root), width=25, **font).grid(columnspan=5)

root.mainloop()